In [1]:
import pandas as pd

data = pd.read_json("folder/data.json")

data.head()

,﻿type,content
0,h2,Page 1
1,h2,Page 2
2,p,"7 (, | /"
3,h2,Page 3
4,h2,Page 4


In [2]:
data.duplicated().sum()

44280

In [3]:
df = data.drop_duplicates()

In [4]:
print(data.shape)
print(df.shape)

(192514, 2)
(148234, 2)


In [5]:
bengali_pattern = r'[\u0980-\u09FF]'

In [6]:
columns_to_check = ['content']

In [7]:
import re

row_has_bengali = df[columns_to_check].apply(
    lambda row: any(bool(re.search(bengali_pattern, str(cell))) for cell in row), axis=1
)

In [8]:
df = df[row_has_bengali]

In [9]:
df.shape

(146134, 2)

In [10]:
df.head()

,﻿type,content
5,p,ঢাকার ইতিহাস।
6,p,ভিভীক্স শব?\n( প্রাচীনকাল হইতে মোসলমানাগমনের প...
7,p,৫৯০৫১
8,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।
9,p,৩


In [11]:
# df = df[['type','content']]

In [12]:
rows_with_nan = df[df.isna().any(axis=1)]

In [13]:
rows_with_nan

,﻿type,content


In [14]:
df = df.dropna()

In [15]:
df.to_csv("folder/data.csv", index=False)

In [16]:
import pandas as pd
import re

# Load the Q&A CSV
df = pd.read_csv("folder/data.csv")

# Only keep the required columns
required_columns = ['type','content']
df = df[required_columns]

# Define Bengali character regex
bengali_pattern = r'[\u0980-\u09FF]'

# Keep rows that have Bengali characters in any of the four fields
row_has_bengali = df[required_columns].apply(
    lambda row: any(bool(re.search(bengali_pattern, str(cell))) for cell in row), axis=1
)
df = df[row_has_bengali]

# Drop rows with any missing values
df = df.dropna()

# Text cleaning function
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r"https?://\S+", "", text)  # Remove URLs
        text = text.replace("\n", " ")  # Remove newlines
        text = text.strip()
    return text

# Apply cleaning only to text columns
for col in required_columns:
    df[col] = df[col].apply(clean_text)

# Save cleaned dataset
cleaned_dataset_path = 'Dataset.csv'
df.to_csv(cleaned_dataset_path, index=False)
print(f"Cleaned dataset saved to {cleaned_dataset_path}")


Cleaned dataset saved to Dataset.csv


In [17]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv("Dataset.csv")

# Keep only required columns
columns = ['type','content']
df = df[columns]

# Regex patterns
bengali_pattern = r'[\u0980-\u09FF]'
emoji_pattern = r'[^\w\s\u0980-\u09FF.,!?ঃঁৎৗৄৢৣ]"'  # Keeps mostly Bengali and punctuation
url_pattern = r"https?://\S+|www\.\S+"
latin_pattern = r'[A-Za-z]'
repeated_punct = r'[!?।:]{2,}'

# Cleaning function
def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(url_pattern, "", text)  # Remove URLs
    text = re.sub(emoji_pattern, "", text)  # Remove emojis and odd chars
    text = re.sub(latin_pattern, "", text)  # Remove Latin characters
    text = re.sub(repeated_punct, ".", text)  # Normalize repeated punctuation
    text = text.replace("\n", " ").replace("\r", " ")  # Remove newlines
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip()

# Apply cleaning
for col in columns:
    df[col] = df[col].apply(clean_text)

# Filter: must contain sufficient Bengali text
def is_valid_bengali(text, min_ratio=0.6):
    total_chars = len(text)
    bengali_chars = len(re.findall(bengali_pattern, text))
    return total_chars > 0 and (bengali_chars / total_chars) >= min_ratio

# Drop rows that fail any validation
df = df[
    df['type'].apply(lambda x: is_valid_bengali(x) and len(x) >= 10) &
    df['content'].apply(lambda x: is_valid_bengali(x) and len(x) >= 10)
]

# Drop rows with NaNs
df = df.dropna()

# Save cleaned dataset
output_path = "data.csv"
df.to_csv(output_path, index=False)
print(f"Strictly cleaned dataset saved to {output_path}")

Strictly cleaned dataset saved to data.csv


In [18]:
df1 = pd.read_csv("Dataset.csv")
df2 = pd.read_csv("data.csv")

print(df1)
print(df2)
print("--------------------------------")
print(df1.shape)
print(df2.shape)

       type                                            content
0         p                                      ঢাকার ইতিহাস।
1         p  ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...
2         p                                              ৫৯০৫১
3         p                     স্রীযতীন্দ্র মোহন রায় প্রণীত।
4         p                                                  ৩
...     ...                                                ...
146129    p  ↑ "জাবালে নূরের মালিক-চালকের বিরুদ্ধে অভিযোগপত...
146130    p  ↑ "জাবালে নূর বাস মালিকের হাইকোর্টে জামিন"। কা...
146131    p  বহিঃসংযোগ[সম্পাদনা] নিরাপদ সড়ক আন্দোলনের বিশদ...
146132    p  ইউটিউবে Students take to streets for 7th day, ...
146133    p  এ পৃষ্ঠায় শেষ পরিবর্তন হয়েছিল ০২:১৯টার সময়,...

[146134 rows x 2 columns]
Empty DataFrame
Columns: [type, content]
Index: []
--------------------------------
(146134, 2)
(0, 2)


In [19]:
df1.head()

,type,content
0,p,ঢাকার ইতিহাস।
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...
2,p,৫৯০৫১
3,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।
4,p,৩


In [20]:
df1.isnull().sum()

type       0
content    0
dtype: int64

In [21]:
import pandas as pd
import re

df = pd.read_csv("Dataset.csv")

In [22]:
# Bengali digits
bengali_digits = '০১২৩৪৫৬৭৮৯'
only_bengali_digits_pattern = f'^[{bengali_digits}]+$'

# Function: Check if content is made ONLY of Bengali numbers (no other text/symbols)
def is_only_bengali_number(text):
    return bool(re.fullmatch(only_bengali_digits_pattern, str(text).strip()))

# Remove rows from df where 'content' is only Bengali numbers
df_cleaned = df[~df['content'].apply(is_only_bengali_number)].copy()


In [23]:
df.head()

,type,content
0,p,ঢাকার ইতিহাস।
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...
2,p,৫৯০৫১
3,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।
4,p,৩


In [24]:
df_cleaned.head()

,type,content
0,p,ঢাকার ইতিহাস।
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...
3,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।
5,p,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...
6,p,১৩২২ বঙ্গাব । ্রন্থকার়ের সর্বস্বত্ব সংরক্ষিত।...


In [25]:
print(df.shape)
print(df_cleaned.shape)

(146134, 2)
(144995, 2)


In [26]:
df_cleaned.to_csv("cleaned_data.csv", index=False)

In [27]:
dft = pd.read_csv("Dataset.csv")

dft.head()

,type,content
0,p,ঢাকার ইতিহাস।
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...
2,p,৫৯০৫১
3,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।
4,p,৩


In [28]:
dft = pd.read_csv("cleaned_data.csv",)

dft.head()

,type,content
0,p,ঢাকার ইতিহাস।
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...
2,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।
3,p,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...
4,p,১৩২২ বঙ্গাব । ্রন্থকার়ের সর্বস্বত্ব সংরক্ষিত।...


In [29]:
dft.shape

(144995, 2)

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('cleaned_data.csv')

df_train, df_temp = train_test_split(df, test_size=0.03, random_state=42, shuffle=True)

df_train.to_csv("sData/df_train.csv", index=False)
df_temp.to_csv("sData/df_temp.csv", index=False)


In [33]:
df_train.shape

(140645, 2)

In [34]:
df_temp.shape

(4350, 2)

In [42]:
data = pd.read_csv("preprocessed_bengali_data.csv")

data.head()

,type,content,title,context,label,class
0,p,"২৮ ক্রায়ার, ২য় খণ্ড, পৃ. ১১৯।",<extra_id_0>. ) <extra_id_34>,<extra_id_0>. ) <extra_id_34>.খণ্ড.,neutral,news
1,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,<extra_id_0>।' বিষয়ের তিন পুত--,<extra_id_0>।' বিষয়ের তিন পুত--,neutral,news
2,p,২২ ঢাকার ইতিহাস। [২য়খণ্ড,<extra_id_0>। <extra_id_10>ুন,<extra_id_0>। <extra_id_10>ুন বিস্তারিত.: <ext...,neutral,history
3,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,"<extra_id_0>।,।","<extra_id_0>।,।,..)।",neutral,news
4,p,মণিপুরের ইতিহাস। ৭৯,<extra_id_0> । ) <extra_id_34> ।,<extra_id_0> । ) <extra_id_34> । ২৪.,neutral,history


In [44]:
data['label'].value_counts()

label
neutral    4350
Name: count, dtype: int64

In [45]:
data['class'].value_counts()

class
news          3636
history        467
philosophy     182
text_book       56
story            9
Name: count, dtype: int64

In [47]:
data = pd.read_csv("preprocessed_data.csv")

data.head()

,type,content,title,context,label,class
0,p,"২৮ ক্রায়ার, ২য় খণ্ড, পৃ. ১১৯।","২৮ ক্রায়ার, ২য় খণ্ড, পৃ. ১১৯","২৮ ক্রায়ার, ২য় খণ্ড, পৃ. ১১৯।",neutral,news
1,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ। ' সাভাইয...,positive,news
2,p,২২ ঢাকার ইতিহাস। [২য়খণ্ড,২২ ঢাকার ইতিহাস,২২ ঢাকার ইতিহাস।,positive,history
3,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,positive,news
4,p,মণিপুরের ইতিহাস। ৭৯,মণিপুরের ইতিহাস,মণিপুরের ইতিহাস।,positive,history


In [48]:
data['label'].value_counts()

label
negative    2165
positive    1521
neutral      664
Name: count, dtype: int64

In [49]:
data['class'].value_counts()

class
news          3636
history        467
philosophy     182
text_book       56
story            9
Name: count, dtype: int64

In [51]:
data = pd.read_csv("processed_bengali_qa.csv")

data.head(20)

,id,type,content,question,answer
0,2,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,এই লেখার মূল বিষয়বস্তু কী?,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...
1,2,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,এতে কী ঐতিহাসিক ঘটনা বর্ণনা করা হয়েছে?,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...
2,2,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,লেখাটি কোন সময়কাল নিয়ে আলোচনা করে?,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...
3,2,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,এতে উল্লেখযোগ্য ব্যক্তি বা ঘটনাবলি কী কী?,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...
4,2,p,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...,এর প্রেক্ষাপট কী ছিল?,সাভাই হইতে এই বংণের গণনা হইয়া থাকেণ।' সাভাইয়...
5,4,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,এই লেখার মূল বিষয়বস্তু কী?,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...
6,4,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,এতে কী ঐতিহাসিক ঘটনা বর্ণনা করা হয়েছে?,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...
7,4,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,লেখাটি কোন সময়কাল নিয়ে আলোচনা করে?,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...
8,4,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,এতে উল্লেখযোগ্য ব্যক্তি বা ঘটনাবলি কী কী?,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...
9,4,p,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...,এর প্রেক্ষাপট কী ছিল?,বিদ্যাপতি ৮-ত্বঙ্টীঘ পঞ্চদশ শত্বাবীব প্রারত্তে...


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('sData/df_temp.csv')

df_train, df_temp = train_test_split(df, test_size=0.03, random_state=42, shuffle=True)

df_train.to_csv("sData/temp1.csv", index=False)
df_temp.to_csv("sData/temp2.csv", index=False)


In [3]:
print(df_train.shape)
print(df_temp.shape)

(4219, 2)
(131, 2)


In [6]:
data = pd.read_csv("bengali_llm_qa_output.csv")

data.head()

,id,type,content,question,answer
0,1,p,"কোন ণতাত্বিকের কাছে ব্যাপারটা অদ্ভুত ধংনের, যে...",NaN,NaN
1,2,p,প্রথম পানৎসার ডিভিশনের আরুমণ সাফল্যমাওত হয় । ...,NaN,NaN
2,3,p,সুজ্যেয় শ্যাম,NaN,NaN
3,4,p,এই কথা বলতে বলতে তার আত্মা-পাখী উড়ে গেল। আর অ...,NaN,NaN
4,5,p,শ্রযুক্ত বাবু শ্রপৎসিংহ দুগর অনেক সভানমিতির সভ...,NaN,NaN


In [7]:
data['question'].value_counts()

question
পণ্য: ৮৯০,৬৯২ টাকা ৫১০৫,৬৭৫,৬৯৮ টাকা,... ১১৮১২৪১৩৯০৬ টাকা, ৫৫৫৫,৫৭৯ টাকা, ৩৩৩ টাকা ৫,৫৬৫,৫৭৯ বুগতা                                                               3
এই পাঠ্যাংশ অনুসারে, একটা প্রশ্ন করুন: ১ ৪০০+৮, ২০০০০০% ০৯০৯১০৫ ১০,০৪১০১০১০. ২. ২। ৩.০.৭১৮০৫: ১০১০, ০.২।                                                         3
এর মধ্যে উল্লেখযোগ্য ছিল বরিশালের সপ্তক  পরিচয়, সংহতি, হাগুরার "অভিবাদ" ইত্যাদি। ১৯৪৪ সালের জুলাই-আন্তের ফ্যাসিবাদ বিরোধী পত্রিকাগুলিও ছিল বাংলায় প্রকাশিত।    2
এই অনুচ্ছেদ থেকে প্রশ্ন করুন: সডলের রচিত উদয়নকুন্ডা কাব্যে অভিনন্দ পালকুলচন্দ্র ও বিক্রমশীল নামে পরিচিত। তাঁর পিতার নাম বিক্রমশীল।                              2
এই প্রবন্ধ অনুসারে একটি প্রশ্ন করুন: চৌর্য [ রণিকাবচি] নান ট্রাক্স...অকিঞ্চিৎ প্রত্যাবর্তন...।                                                                   2
এ ছাড়া, এই পাঠ্যাংশ অনুসারে, গির্জাকে একটা প্রশ্ন দিতে হতো: (৬%৮৬১৫১) অথবা ফসলের এক চতুর্থাংশ (এদ) অথবা কর।                                                     2
এই পাঠ্যাংশ অ

In [8]:
data['question'].isnull().sum()

117

In [9]:
data['answer'].isnull().sum()

117

In [10]:
data = pd.read_csv("bengali_qa.csv")

data.head()

,id,type,content,question,answer
0,1,p,"কোন ণতাত্বিকের কাছে ব্যাপারটা অদ্ভুত ধংনের, যে...",এই পাঠ্যাংশ থেকে একটি প্রশ্ন তৈরি করুন: কোন তত...,এই লেখার উপর ভিত্তি করে একটি প্রশ্ন তৈরি করুন:...
1,2,p,প্রথম পানৎসার ডিভিশনের আরুমণ সাফল্যমাওত হয় । ...,দুপুরের মধ্যে আমরা সেই জায়গা ও শহরের দিকে তাক...,এই লেখার ভিত্তিতে আমরা উত্তর দিয়েছিলাম: আমরা ...
2,4,p,এই কথা বলতে বলতে তার আত্মা-পাখী উড়ে গেল। আর অ...,এই পাঠ্যাংশ থেকে একটা প্রশ্ন তৈরি করুন: তার আত...,প্রশ্ন: এই পাঠ্যাংশ থেকে একটা প্রশ্ন তৈরি করুন...
3,5,p,শ্রযুক্ত বাবু শ্রপৎসিংহ দুগর অনেক সভানমিতির সভ...,এই পাঠ্যাংশ থেকে একটি প্রশ্ন তৈরি করুন: শ্রীজি...,এই লেখার ভিত্তিতে একটি প্রশ্ন: শ্রীজিত বাবু শ্...
4,6,p,সন্ধ্যা ক'ছাক। 5 সময়ে যুগ শুক হলে বিউগলের অ ম...,"৯ শন বিবদা ও গোলাগুলি ম্বাশ, যখন শত্রুপক্ষের ব...","৫ শন বিবদা এবং গোলাগুলি ম্বাশ, যখন শত্রুপক্ষের..."


In [11]:
data['question'].value_counts()

question
এই পাঠ্যাংশ থেকে একটি প্রশ্ন তৈরি করুন: কোন তত্ত্বীয়ভাবে এটি অদ্ভুত, যেমন  জার্মান বামপন্থী গণতন্ত্রীরা বলেন যে সাম্রাজ্যিক এবং যুদ্ধ যা জন্ম নিয়েছে তা অ-আগ্রহী, তাই এটি পুঁজিবাদের একটি অবশ্যম্ভাবী ক্রোধ, যা লগমাপুাজর কর্তৃত্বকে গ্রহণ করে।                                                            1
এই পাঠ্যাংশ থেকে একটা প্রশ্ন তৈরি করুন: বনের সভ্য রাজার ছবি মনে গেঁথে যায়, সকালে গারদা দিয়ে, যেভাবে তিনি স্সানাহার ব্যায়াম করেন।                                                                                                                                                                          1
এই পাঠ্যাংশ থেকে একটি প্রশ্ন তৈরি করুন: ১৯৩৪ সালে রাশিয়ার প্রথম পাঁচ-বারের মতো পরিকল্পনা সম্পন্ন হয়েছে - - পরিস্থিতি এখন উত্তপ্ত,  দেশ,  এর উপর নির্মম হত্যা এবং নির্যাতন শুরু করেছে।                                                                                                                      1
তার সামনে, সে একটা বড় বরফের ওপর দিয়ে দৌড়ে গেল, হু তুং-শেং, দেখে মনে হলো সে বরফে

In [12]:
data['question'].isnull().sum()

0

In [13]:
data['answer'].isnull().sum()

0

In [3]:
import pandas as pd

fdata = pd.read_csv("final_data.csv")

fdata.head()

,type,content,title,context,label,class
0,p,ঢাকার ইতিহাস।,ঢাকার ইতিহাস,ঢাকার ইতিহাস।,positive,history
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...,neutral,news
2,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।,স্রীযতীন্দ্র মোহন রায় প্রণীত,স্রীযতীন্দ্র মোহন রায় প্রণীত।,positive,news
3,p,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...,positive,news
4,p,১৩২২ বঙ্গাব । ্রন্থকার়ের সর্বস্বত্ব সংরক্ষিত।...,১৩২২ বঙ্গাব,১৩২২ বঙ্গাব। ্রন্থকার়ের সর্বস্বত্ব সংরক্ষিত।,positive,news


In [4]:
fdata['label'].value_counts()

label
negative    72297
positive    50552
neutral     22146
Name: count, dtype: int64

In [5]:
fdata['class'].value_counts()

class
news          120093
history        16438
philosophy      6376
text_book       1488
story            600
Name: count, dtype: int64

In [6]:
import pandas as pd

fdata = pd.read_csv("filtered_dataset.csv")

fdata.head()

,type,content,title,context,label,class
0,p,ঢাকার ইতিহাস।,ঢাকার ইতিহাস,ঢাকার ইতিহাস।,positive,history
1,p,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...,ভিভীক্স শব? ( প্রাচীনকাল হইতে মোসলমানাগমনের পূ...,neutral,news
2,p,স্রীযতীন্দ্র মোহন রায় প্রণীত।,স্রীযতীন্দ্র মোহন রায় প্রণীত,স্রীযতীন্দ্র মোহন রায় প্রণীত।,positive,news
3,p,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...,__কলিকাতাঁ_ ২৯৭ নং আগার চিৎপুর রোড হইতে শ্রীশশ...,positive,news
4,p,১৩২২ বঙ্গাব । ্রন্থকার়ের সর্বস্বত্ব সংরক্ষিত।...,১৩২২ বঙ্গাব,১৩২২ বঙ্গাব। ্রন্থকার়ের সর্বস্বত্ব সংরক্ষিত।,positive,news


In [7]:
import pandas as pd

df = pd.read_json("muktijuddho_qa_alpaca.json")

In [8]:
df.to_csv("muktijuddho_qa_alpaca.csv")